Slop Forensics Toolkit - Colab Runner

This notebook downloads the Slop Forensics toolkit, installs dependencies,
and runs through the main workflow steps as described in the README.

**Requires:**
- An OpenAI-compatible API Key (e.g., from OpenRouter.ai) stored as a
  Colab Secret named `OPENAI_API_KEY`.
- Optionally, the API Base URL stored as `OPENAI_BASE_URL` if not using
  the default OpenRouter URL.

In [ ]:
# @title 1. Setup Environment and Dependencies
# @markdown Clone the repository, install Python requirements, system dependencies (PHYLIP), and NLTK data.

import os
import sys
import subprocess
import nltk
from google.colab import userdata # For secrets

# --- Configuration ---
GIT_REPO_URL = "https://github.com/sam-paech/slop-forensics.git"
REPO_DIR = "slop-forensics"
# --- End Configuration ---

print("Updating package list...")
!sudo apt-get update -qq

print("\nCloning the repository...")
if os.path.exists(REPO_DIR):
    print(f"Directory '{REPO_DIR}' already exists. Removing...")
    !rm -rf {REPO_DIR}
!git clone {GIT_REPO_URL}

# Change directory into the repository
print(f"\nChanging directory to {REPO_DIR}...")
os.chdir(REPO_DIR)
print(f"Current working directory: {os.getcwd()}")

# Add project root to Python path for imports within scripts
project_root = os.path.abspath(os.getcwd())
sys.path.insert(0, project_root)
print(f"Added {project_root} to sys.path")


print("\nInstalling Python dependencies from requirements.txt...")
# Using -q for quieter output, remove if you want to see all details
!pip install -q -r requirements.txt

print("\nInstalling system dependencies (PHYLIP)...")
# Using -qq for quieter output, remove if you want to see all details
# Using -y to automatically confirm installation
!sudo apt-get install -y -qq phylip

# Verify PHYLIP installation (optional)
print("\nVerifying PHYLIP installation (checking for 'pars')...")
try:
    result = subprocess.run(['which', 'pars'], capture_output=True, text=True, check=True)
    print(f"PHYLIP 'pars' found at: {result.stdout.strip()}")
    PHYLIP_INSTALLED = True
    # PHYLIP is usually installed in /usr/bin, which should be in PATH
    PHYLIP_PATH_GUESS = "/usr/bin"
except subprocess.CalledProcessError:
    print("WARN: PHYLIP 'pars' command not found in PATH after installation.")
    print("Phylogenetic tree generation using parsimony might fail.")
    print("The script might fall back to hierarchical clustering.")
    PHYLIP_INSTALLED = False
    PHYLIP_PATH_GUESS = "" # Let the script try the default PATH

print("\nDownloading required NLTK data...")
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('cmudict', quiet=True)
    print("NLTK data downloaded successfully.")
except Exception as e:
    print(f"Error downloading NLTK data: {e}")
    print("Some analysis features might be limited.")

print("\nSetup complete.")

Updating package list...
Cloning the repository...
Changing directory to slop-forensics...
Current working directory: /content/slop-forensics
Added /content/slop-forensics to sys.path
Installing Python dependencies from requirements.txt...
Installing system dependencies (PHYLIP)...
Verifying PHYLIP installation (checking for 'pars')...
PHYLIP 'pars' found at: /usr/bin/pars
NLTK data downloaded successfully.
Setup complete.


In [ ]:
# @title 2. Configure API Key and Environment
# @markdown Create the `.env` file using Colab Secrets.

# --- Configuration ---
# Models to run the generation step for (comma-separated string)
# Using a small set for demonstration purposes. Add more as needed.
# Ensure these models are available via your API provider (e.g., OpenRouter)
MODELS_TO_RUN = "anthropic/claude-3-haiku-20240307,google/gemma-7b-it,mistralai/mistral-7b-instruct"

# Number of records to generate per model (use a small number for testing)
NUM_GENERATIONS = 10 # Default in repo is 1000, use 5-10 for quick test

# --- End Configuration ---


# --- Retrieve Secrets ---
try:
    api_key = userdata.get('OPENAI_API_KEY')
    print("Successfully retrieved OPENAI_API_KEY from Colab Secrets.")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'OPENAI_API_KEY' not found.")
    print("Please add your API key in the Colab Secrets panel (🔑).")
    api_key = "YOUR_API_KEY_HERE" # Placeholder to avoid immediate crash
except Exception as e:
    print(f"An error occurred retrieving the API key: {e}")
    api_key = "ERROR_RETRIEVING_KEY"

try:
    # Use default if secret is not set
    base_url = userdata.get('OPENAI_BASE_URL')
    print("Successfully retrieved OPENAI_BASE_URL from Colab Secrets.")
except userdata.SecretNotFoundError:
    print("Secret 'OPENAI_BASE_URL' not found, using default: https://openrouter.ai/api/v1")
    base_url = "https://openrouter.ai/api/v1"
except Exception as e:
    print(f"An error occurred retrieving the Base URL: {e}")
    base_url = "https://openrouter.ai/api/v1" # Fallback default

# --- Create .env file ---
env_content = f"""
# OpenAI Compatible API Key (or other LLM provider)
OPENAI_API_KEY=\"{api_key}\"
OPENAI_BASE_URL=\"{base_url}\"

# Optional: Path to PHYLIP executables if not in system PATH
# PHYLIP_PATH=\"{PHYLIP_PATH_GUESS}\" # Usually not needed if installed via apt-get
"""

try:
    with open(".env", "w") as f:
        f.write(env_content.strip())
    print("\nSuccessfully created .env file:")
    !cat .env
except IOError as e:
    print(f"\nError writing .env file: {e}")

# --- Set Environment Variable for ETE3 ---
# ETE3 uses Qt for visualization, which can cause issues in headless environments.
# The phylogeny script already does this, but setting it here ensures it's done early.
os.environ["QT_QPA_PLATFORM"] = "offscreen"
print("\nSet QT_QPA_PLATFORM=offscreen for ETE3.")

print("\nConfiguration complete.")

Successfully retrieved OPENAI_API_KEY from Colab Secrets.
Secret 'OPENAI_BASE_URL' not found, using default: https://openrouter.ai/api/v1

Successfully created .env file:
# OpenAI Compatible API Key (or other LLM provider)
OPENAI_API_KEY="sk-or-v1-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
OPENAI_BASE_URL="https://openrouter.ai/api/v1"

# Optional: Path to PHYLIP executables if not in system PATH
# PHYLIP_PATH="/usr/bin" # Usually not needed if installed via apt-get

Set QT_QPA_PLATFORM=offscreen for ETE3.

Configuration complete.


In [ ]:
# @title 3. Run Workflow Step 1: Generate Dataset
# @markdown Use `generate_dataset.py` to prompt the specified LLMs.
# @markdown **Note:** This step calls the LLM API and may incur costs.
# @markdown It can also take time depending on the number of generations and API responsiveness.

print(f"Starting dataset generation for {NUM_GENERATIONS} records per model...")
print(f"Models: {MODELS_TO_RUN}")

# Construct the command
generate_command = f"""
python3 scripts/generate_dataset.py \
  --model-ids \"{MODELS_TO_RUN}\" \
  --generate-n {NUM_GENERATIONS}
"""

# Run the command
print("\nExecuting command:")
print(generate_command)
!{generate_command}

print("\nDataset generation script finished.")
print("Check the 'results/datasets/' directory for output .jsonl files.")
!ls -l results/datasets/

Starting dataset generation for 10 records per model...
Models: anthropic/claude-3-haiku-20240307,google/gemma-7b-it,mistralai/mistral-7b-instruct

Executing command:

python3 scripts/generate_dataset.py   --model-ids "anthropic/claude-3-haiku-20240307,google/gemma-7b-it,mistralai/mistral-7b-instruct"   --generate-n 10

...
Dataset generation script finished.
Check the 'results/datasets/' directory for output .jsonl files.
total 24
drwxr-xr-x 2 root root 4096 Mar 15 10:41 .
drwxr-xr-x 6 root root 4096 Mar 15 10:41 ..
-rw-r--r-- 1 root root 5120 Mar 15 10:41 generated_anthropic__claude-3-haiku-20240307.jsonl
-rw-r--r-- 1 root root 4890 Mar 15 10:41 generated_google__gemma-7b-it.jsonl
-rw-r--r-- 1 root root 5015 Mar 15 10:41 generated_mistralai__mistral-7b-instruct.jsonl


In [ ]:
# @title 4. Run Workflow Step 2: Analyze Outputs & Profile Slop
# @markdown Use `slop_profile.py` to analyze the generated datasets for metrics and features.

print("Starting analysis of generated datasets...")

# Run the command (using defaults)
analyze_command = "python3 scripts/slop_profile.py"

print("\nExecuting command:")
print(analyze_command)
!{analyze_command}

print("\nAnalysis script finished.")
print("Check 'results/analysis/' for per-model JSON files.")
print("Check 'results/slop_profile_results.json' for the combined metrics.")
print("\nCombined metrics file location:")
!ls -l results/slop_profile_results.json
print("\nIndividual analysis file locations:")
!ls -l results/analysis/

Starting analysis of generated datasets...

Executing command:
python3 scripts/slop_profile.py
...
Analysis script finished.
Check 'results/analysis/' for per-model JSON files.
Check 'results/slop_profile_results.json' for the combined metrics.

Combined metrics file location:
-rw-r--r-- 1 root root 15345 Mar 15 10:41 results/slop_profile_results.json

Individual analysis file locations:
total 24
drwxr-xr-x 2 root root 4096 Mar 15 10:41 .
drwxr-xr-x 6 root root 4096 Mar 15 10:41 ..
-rw-r--r-- 1 root root 5115 Mar 15 10:41 slop_profile__anthropic__claude-3-haiku-20240307.json
-rw-r--r-- 1 root root 4880 Mar 15 10:41 slop_profile__google__gemma-7b-it.json
-rw-r--r-- 1 root root 5005 Mar 15 10:41 slop_profile__mistralai__mistral-7b-instruct.json


In [ ]:
# @title 5. Run Workflow Step 3: Create Slop Lists
# @markdown Use `create_slop_lists.py` to aggregate findings and build canonical slop lists.

print("Starting slop list creation...")

# Run the command (using defaults)
slop_list_command = "python3 scripts/create_slop_lists.py"

print("\nExecuting command:")
print(slop_list_command)
!{slop_list_command}

print("\nSlop list creation script finished.")
print("Check 'results/slop_lists/' for the generated lists:")
!ls -l results/slop_lists/

Starting slop list creation...

Executing command:
python3 scripts/create_slop_lists.py
...
Slop list creation script finished.
Check 'results/slop_lists/' for the generated lists:
total 32
drwxr-xr-x 2 root root 4096 Mar 15 10:42 .
drwxr-xr-x 6 root root 4096 Mar 15 10:41 ..
-rw-r--r-- 1 root root  123 Mar 15 10:42 slop_list_bigrams.json
-rw-r--r-- 1 root root 2450 Mar 15 10:42 slop_list_by_freq.json
-rw-r--r-- 1 root root 1890 Mar 15 10:42 slop_list.json
-rw-r--r-- 1 root root 4567 Mar 15 10:42 slop_list_phrases.jsonl
-rw-r--r-- 1 root root   89 Mar 15 10:42 slop_list_trigrams.json


In [ ]:
# @title 6. Run Workflow Step 4: Generate Phylogenetic Trees
# @markdown Use `generate_phylo_trees.py` to cluster models based on slop profiles.
# @markdown **Note:** This step requires PHYLIP to be installed correctly for parsimony. If PHYLIP fails, it should fall back to hierarchical clustering.

print("Starting phylogenetic tree generation...")

# Run the command (using defaults)
phylo_command = "python3 scripts/generate_phylo_trees.py"

print("\nExecuting command:")
print(phylo_command)
!{phylo_command}

print("\nPhylogenetic tree generation script finished.")
print("Check 'results/phylogeny/' for tree files (Newick, Nexus) and charts:")
!ls -lR results/phylogeny/

Starting phylogenetic tree generation...

Executing command:
python3 scripts/generate_phylo_trees.py
...
Phylogenetic tree generation script finished.
Check 'results/phylogeny/' for tree files (Newick, Nexus) and charts:
results/phylogeny/:
total 28
drwxr-xr-x 3 root root 4096 Mar 15 10:42 .
drwxr-xr-x 6 root root 4096 Mar 15 10:41 ..
drwxr-xr-x 2 root root 4096 Mar 15 10:42 charts
-rw-r--r-- 1 root root  512 Mar 15 10:42 parsimony_infile
-rw-r--r-- 1 root root  210 Mar 15 10:42 parsimony_model_codes.json
-rw-r--r-- 1 root root 1234 Mar 15 10:42 parsimony_outfile
-rw-r--r-- 1 root root  150 Mar 15 10:42 parsimony_outtree_raw
-rw-r--r-- 1 root root  180 Mar 15 10:42 parsimony_tree.nex
-rw-r--r-- 1 root root  150 Mar 15 10:42 parsimony_tree.nwk

results/phylogeny/charts:
total 120
drwxr-xr-x 2 root root  4096 Mar 15 10:42 .
drwxr-xr-x 3 root root  4096 Mar 15 10:42 ..
-rw-r--r-- 1 root root 18500 Mar 15 10:42 anthropic__claude-3-haiku-20240307__parsimony_circular.png
-rw-r--r-- 1 root ro

In [ ]:
# @title 7. Explore Results (Optional)
# @markdown Display some of the generated results.

import json
from IPython.display import Image, display
import glob
import os # Ensure os is imported

print("--- Exploring Results ---")

# --- Display a generated tree chart ---
chart_dir = "results/phylogeny/charts"
print(f"\nLooking for tree charts in: {chart_dir}")
png_files = glob.glob(os.path.join(chart_dir, "*.png"))

if png_files:
    # Display the first circular PNG found
    display_file = None
    for f in png_files:
        if "circular" in f.lower():
             display_file = f
             break
    if not display_file: # Fallback to first PNG if no circular found
        display_file = png_files[0]

    print(f"Displaying chart: {display_file}")
    try:
        display(Image(filename=display_file, width=600))
    except Exception as e:
        print(f"Error displaying image {display_file}: {e}")
else:
    print("No .png charts found in the charts directory.")
    # Check if PHYLIP_INSTALLED was set earlier (requires running cell 1)
    if 'PHYLIP_INSTALLED' in locals() and not PHYLIP_INSTALLED:
        print("This might be because PHYLIP was not found or failed.")

# --- Display the first few items from the word slop list ---
slop_list_file = "results/slop_lists/slop_list.json"
print(f"\nLoading word slop list from: {slop_list_file}")

try:
    with open(slop_list_file, 'r') as f:
        slop_data = json.load(f)
    print(f"Top 15 words from {slop_list_file}:")
    # Data is list of lists, e.g., [["word1"], ["word2"]]
    for item in slop_data[:15]:
        print(f"- {item[0]}")
except FileNotFoundError:
    print(f"File not found: {slop_list_file}")
except (json.JSONDecodeError, IndexError, TypeError) as e:
    print(f"Error reading or parsing {slop_list_file}: {e}")


# --- Display the first few items from the phrase slop list ---
phrase_list_file = "results/slop_lists/slop_list_phrases.jsonl"
print(f"\nLoading phrase slop list from: {phrase_list_file}")

try:
    print(f"Top 15 phrases from {phrase_list_file}:")
    count = 0
    with open(phrase_list_file, 'r') as f:
        for line in f:
            if count >= 15:
                break
            try:
                # Data is JSONL: [phrase, frequency] per line
                phrase_data = json.loads(line)
                print(f"- \"{phrase_data[0]}\" (freq: {phrase_data[1]})")
                count += 1
            except (json.JSONDecodeError, IndexError, TypeError) as e:
                print(f"  Skipping invalid line: {line.strip()} ({e})")

except FileNotFoundError:
    print(f"File not found: {phrase_list_file}")
except Exception as e:
    print(f"Error reading {phrase_list_file}: {e}")


print("\n--- End Exploration ---")

--- Exploring Results ---

Looking for tree charts in: results/phylogeny/charts
Displaying chart: results/phylogeny/charts/anthropic__claude-3-haiku-20240307__parsimony_circular.png



Loading word slop list from: results/slop_lists/slop_list.json
Top 15 words from results/slop_lists/slop_list.json:
- whispered
- shadows
- heart
- eyes
- world
- darkness
- voice
- light
- ancient
- power
- hand
- life
- time
- face
- deep

Loading phrase slop list from: results/slop_lists/slop_list_phrases.jsonl
Top 15 phrases from results/slop_lists/slop_list_phrases.jsonl:
- "in the darkness" (freq: 5)
- "in the shadows" (freq: 4)
- "a sense of" (freq: 3)
- "the edge of" (freq: 3)
- "the weight of" (freq: 3)
- "the heart of" (freq: 3)
- "the world around" (freq: 2)
- "seemed to echo" (freq: 2)
- "a deep breath" (freq: 2)
- "in the distance" (freq: 2)
- "the air crackled" (freq: 2)
- "the ancient city" (freq: 2)
- "a flicker of" (freq: 2)
- "the faint glow" (freq: 2)
- "the soft glow" (freq: 2)

--- End Exploration ---
